In [ ]:
from discovery_utils.getters import gtr
from discovery_utils.utils import search
from discovery_utils.utils import analysis_gtr, analysis, charts, viz_landscape

from src import PROJECT_DIR
from src import VECTOR_DB_DIR

OUTPUT_DIR = PROJECT_DIR / 'data/2025_01_MS_ahl/'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

## Gateway to Research search

In [ ]:
GTR = gtr.GtrGetter(vector_db_path=VECTOR_DB_DIR)

In [ ]:
Search = search.SearchDataset(GTR, GTR.projects_enriched, "config_food.yaml")
search_df = Search.do_search()

In [ ]:
selected_df = search_df.query('_score_avg > 0.25')
matching_ids = set(list(selected_df.id.to_list()))

In [ ]:
selected_df

In [ ]:
selected_df.columns

In [ ]:
ts_df = (
    analysis_gtr.get_timeseries(selected_df, period='year', min_year=2010, max_year=2025, description_column="abstractText")
    .assign(amount = lambda df: df.amount / 1_000_000)
)
fig = charts.ts_bar(
    ts_df,
    variable='n_projects',
    variable_title="Number of projects",
    category_column="_category",
)
charts.configure_plots(fig, chart_title="")

In [ ]:
fig = charts.ts_bar(
    ts_df,
    variable='amount',
    variable_title="Amount, £ millions",
    category_column="_category",
)
charts.configure_plots(fig, chart_title="")

In [ ]:
# write an sql query to achieve id in test_ids
id_condition = "id in ('{}')".format("', '".join(list(matching_ids)))
vectors_df = GTR.VectorDB.vector_db.search().where(id_condition).limit(30000).to_pandas()
len(vectors_df)

In [ ]:
fig, gtr_viz_df = viz_landscape.generate_gtr_landscape(vectors_df, GTR, min_cluster_size=50,  verbose=True)

In [ ]:
save_name = 'food'
output_path = PROJECT_DIR / f'data/2025_01_MS_ahl/_landscape_gtr_{save_name}.html'
gtr_viz_df.to_csv(PROJECT_DIR / f"data/2025_01_MS_ahl/_table_gtr_{save_name}.csv", index=False)
fig.save(str(output_path))